# Hyperparameter tuning

In the previous section, we did not discuss the parameters of random forest
and gradient-boosting. However, there are a couple of things to keep in mind
when setting these.

This notebook gives crucial information regarding how to set the
hyperparameters of both random forest and gradient boosting decision tree
models.

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">For the sake of clarity, no cross-validation will be used to estimate the
testing error. We are only showing the effect of the parameters
on the validation set of what should be the inner cross-validation.</p>
</div>

## Random forest

The main parameter to tune for random forest is the `n_estimators` parameter.
In general, the more trees in the forest, the better the statistical
performance will be. However, it will slow down the fitting and prediction
time. The goal is to balance computing time and statistical performance when
setting the number of estimators when putting such learner in production.

The `max_depth` parameter could also be tuned. Sometimes, there is no need
to have fully grown trees. However, be aware that with random forest, trees
are generally deep since we are seeking to overfit the learners on the
bootstrap samples because this will be mitigated by combining them.
Assembling underfitted trees (i.e. shallow trees) might also lead to an
underfitted forest.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0)

In [4]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    "n_estimators": [10, 20, 30],
    "max_depth": [3, 5, None],
}
grid_search = GridSearchCV(
    RandomForestRegressor(n_jobs=2), param_grid=param_grid,
    scoring="neg_mean_absolute_error", n_jobs=2,
)
grid_search.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_grid.keys()]
columns += ["mean_test_score", "rank_test_score"]
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results["mean_test_score"] = -cv_results["mean_test_score"]
cv_results[columns].sort_values(by="rank_test_score")
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.114995,0.009190,0.009202,0.000751,3,10,"{'max_depth': 3, 'n_estimators': 10}",-57.053072,-58.597543,-57.800953,-56.302221,-56.377458,57.226250,0.873498,9
1,0.214003,0.006692,0.011596,0.001356,3,20,"{'max_depth': 3, 'n_estimators': 20}",-56.114301,-58.379012,-57.611873,-55.796353,-57.423307,57.064969,0.966165,7
2,0.315399,0.013982,0.012401,0.001020,3,30,"{'max_depth': 3, 'n_estimators': 30}",-56.795244,-58.360240,-57.675368,-55.595180,-57.098016,57.104810,0.924825,8
3,0.165599,0.006651,0.008600,0.000490,5,10,"{'max_depth': 5, 'n_estimators': 10}",-47.771095,-50.148158,-49.530197,-48.060286,-48.718490,48.845645,0.889131,6
4,0.329000,0.007459,0.010801,0.000747,5,20,"{'max_depth': 5, 'n_estimators': 20}",-47.463595,-49.881751,-49.425932,-48.484807,-48.158451,48.682907,0.870670,5
5,0.503401,0.015906,0.013999,0.001266,5,30,"{'max_depth': 5, 'n_estimators': 30}",-47.566988,-49.791238,-49.260329,-48.009273,-47.973601,48.520286,0.852263,4
6,0.479598,0.012811,0.011999,0.001097,None,10,"{'max_depth': None, 'n_estimators': 10}",-35.867558,-36.525082,-36.616488,-35.935589,-36.145070,36.217957,0.303637,3
7,0.919997,0.010991,0.017801,0.000750,None,20,"{'max_depth': None, 'n_estimators': 20}",-35.256576,-35.265678,-35.323813,-34.457095,-34.563372,34.973307,0.380289,2
8,1.366801,0.035571,0.023400,0.000800,None,30,"{'max_depth': None, 'n_estimators': 30}",-33.915970,-34.911443,-35.463030,-34.104623,-33.879657,34.454945,0.627503,1


We can observe that in our grid-search, the largest `max_depth` together
with the largest `n_estimators` led to the best statistical performance.

## Gradient-boosting decision trees

For gradient-boosting, parameters are coupled, so we cannot set the
parameters one after the other anymore. The important parameters are
`n_estimators`, `max_depth`, and `learning_rate`.

Let's first discuss the `max_depth` parameter.
We saw in the section on gradient-boosting that the algorithm fits the error
of the previous tree in the ensemble. Thus, fitting fully grown trees will
be detrimental.
Indeed, the first tree of the ensemble would perfectly fit (overfit) the data
and thus no subsequent tree would be required, since there would be no
residuals.
Therefore, the tree used in gradient-boosting should have a low depth,
typically between 3 to 8 levels. Having very weak learners at each step will
help reducing overfitting.

With this consideration in mind, the deeper the trees, the faster the
residuals will be corrected and less learners are required. Therefore,
`n_estimators` should be increased if `max_depth` is lower.

Finally, we have overlooked the impact of the `learning_rate` parameter
until now. When fitting the residuals, we would like the tree
to try to correct all possible errors or only a fraction of them.
The learning-rate allows you to control this behaviour.
A small learning-rate value would only correct the residuals of very few
samples. If a large learning-rate is set (e.g., 1), we would fit the
residuals of all samples. So, with a very low learning-rate, we will need
more estimators to correct the overall error. However, a too large
learning-rate tends to obtain an overfitted ensemble,
similar to having a too large tree depth.

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    "n_estimators": [10, 30, 50],
    "max_depth": [3, 5, None],
    "learning_rate": [0.1, 1],
}
grid_search = GridSearchCV(
    GradientBoostingRegressor(), param_grid=param_grid,
    scoring="neg_mean_absolute_error", n_jobs=2
)
grid_search.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_grid.keys()]
columns += ["mean_test_score", "rank_test_score"]
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results["mean_test_score"] = -cv_results["mean_test_score"]
cv_results[columns].sort_values(by="rank_test_score")
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.265198,0.008422,0.002802,0.000401,0.1,3,10,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-62.364708,-63.018525,-62.717368,-61.494593,-61.756145,62.270268,0.571787,18
1,0.770600,0.022721,0.004208,0.000399,0.1,3,30,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-45.702805,-45.658183,-46.333430,-45.422513,-45.663489,45.756084,0.305154,11
2,1.256007,0.008301,0.004799,0.000401,0.1,3,50,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-40.487300,-40.566135,-41.350029,-39.726453,-40.878545,40.601692,0.532363,8
3,0.424797,0.017061,0.003004,0.000008,0.1,5,10,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-56.193186,-56.849927,-57.125894,-55.503329,-56.105335,56.355534,0.574994,17
4,1.290594,0.022702,0.005804,0.000403,0.1,5,30,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-39.072257,-39.230976,-40.373662,-38.671351,-39.462528,39.362155,0.567746,5
5,2.152998,0.021054,0.007609,0.000496,0.1,5,50,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-35.660472,-35.559785,-35.950855,-35.278674,-35.777638,35.645485,0.224871,1
6,1.312402,0.017751,0.012394,0.001016,0.1,None,10,"{'learning_rate': 0.1, 'max_depth': None, 'n_e...",-50.522126,-52.479864,-52.346886,-53.133086,-51.511882,51.998769,0.900760,16
7,3.923599,0.035425,0.030592,0.001748,0.1,None,30,"{'learning_rate': 0.1, 'max_depth': None, 'n_e...",-43.819124,-46.045101,-45.143224,-47.325040,-45.711275,45.608753,1.145953,10
8,6.545005,0.080264,0.050996,0.002960,0.1,None,50,"{'learning_rate': 0.1, 'max_depth': None, 'n_e...",-44.276096,-46.195607,-45.380421,-47.647399,-45.839419,45.867788,1.099663,12
9,0.254803,0.004206,0.003000,0.000014,1,3,10,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",-41.792262,-41.845149,-41.636437,-41.129701,-41.607325,41.602175,0.252824,9


<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Here, we tune the <tt class="docutils literal">n_estimators</tt> but be aware that using early-stopping as
in the previous exercise will be better.</p>
</div>